In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import h5py
import os
from tqdm import tqdm
import healpy as hp


from holodeck import plot, detstats, utils
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo

Set up

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

MEDIAN=False
AVG=True

figloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/lunchtalks'


In [ ]:
cmap_Blues = plot.truncate_colormap('Blues', 0.4, 1)
cmap_Greens = plot.truncate_colormap('Greens', 0.4, 1)
cmap_Oranges = plot.truncate_colormap('Oranges', 0.4, 1)
cmap_Greys = plot.truncate_colormap('Greys', 0.4, 0.9)

greys = cmap_Greys(np.linspace(0,1,3))

# hard time both

In [ ]:
col0p0 = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]

gnuplot_map = plot.truncate_colormap('gnuplot_r', 0.05, 0.9)
colors = gnuplot_map(np.linspace(0,1,5))
col3 = mpl.colors.to_rgba('#9f1d93')
colors[3] = col3
colors = np.flip(colors, axis=0)

In [ ]:
target='hard_time'

fig, axs = plot.figax(xscale='linear', nrows=2, 
                      figsize=(3,4.5))
xlabel='Binary Lifetime $\\tau_f$ (Gyr)'
ylabels = [plot.LABEL_EVSS, '$C_{\ell>0}/C_0$']

axs[1].set_xlabel(xlabel)
for ii, ax in enumerate(axs):
    ax.set_ylabel(ylabels[ii])

# n_ss
ax = axs[0]
xx, yy = detstats.get_ratio_arrays(target, bgl=1)
yy *= 0.5
for pp in [50, 95]:
            med, *conf = np.percentile(yy, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx, med, alpha=0.9, color=col0p0[-1])
            ax.fill_between(xx, *conf, color=col0p0[-1], alpha=0.25)
ax.set_yticks([10**-2, 10**-1, 10**0])

# C1C0
ax = axs[1]
labels_fobs=[]
handles=[]
freq_idx = [0,1,2,3,4]
fobs_cents, _ = utils.pta_freqs()
xx, yy, _ = detstats.get_anis_var_arrays(target)
for ff, fidx in enumerate(freq_idx):
    labels_fobs.append(f"{fobs_cents[fidx]*YR:.2f}")

    for pp in [68,]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy[:,fidx,:], [50, 50-pp/2, 50+pp/2], axis=-1)
        # if ff%2 == 0: 
        ax.fill_between(xx, *conf, color=colors[ff], alpha=0.25, zorder=6-ii)
    hh, = ax.plot(xx, med, alpha=0.9, color=colors[ff], zorder=12-ii)
    handles.append(hh)

    ax.axhline(Cl_nihan[ff,0], linestyle='--', color='tab:blue')

fig.subplots_adjust(hspace=0)
leg = fig.legend(handles=handles, labels=labels_fobs, bbox_to_anchor = (0.91, 0.29), loc='center left',
           ncols = 1, title='$f\ (\mathrm{yr}^{-1})$ ', title_fontsize=12, 
           columnspacing=1.25,)

filename = figloc+f'/{target}_evss_and_c1c0.png'
fig.savefig(filename, transparent=False, facecolor=fig.get_facecolor(),
            bbox_inches='tight')

# Single Source hc Map

In [ ]:
target = 'hard_time'
data, params, dsdat = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bgl=1, nexcl=0,
        )
hc_ss = data[10]['hc_ss']
hc_bg = data[10]['hc_bg']
gamma_ssi = dsdat[10]['gamma_ssi']

In [ ]:


fig, ax = plot.figax_single(xlabel=plot.LABEL_GW_FREQUENCY_YR, 
                     )
ax.set_ylabel('Characteristic Strain $h_c$')
xx = data[10]['fobs_cents'] *YR
yy = xx**(-2/3)*1.1*10**-15

_, bg = holo.single_sources.resample_loudest(hc_ss, hc_bg, 1)
ax.plot(xx,yy, color='k', alpha=0.25, linestyle='dashed')

ax.plot(xx, np.median(bg, axis=-1), color='k')
for pp in [50,95]:
    lo, hi = np.percentile(bg, (50-pp/2, 50+pp/2), axis=-1)
    ax.fill_between(xx, lo, hi, color='k', alpha=0.2)


# ax.scatter(xx, np.max(hc_ss[:,0,:], axis=-1), color='k', alpha=0.2)

nreals = 5
nloudest = 3
cmap = plot.truncate_colormap('jet_r',0.1, 0.999)
colors = cmap(np.linspace(0,1,nreals))
colors = ['r',  'dodgerblue',
          'purple', 'g', 'orange']

seed = 7197
# seed = 89118
seed = 1811
# seed=None
if seed is None:
    seed = np.random.randint(99999)   # get a random number
    print(f"random seed: {seed}")                           # print it out so we can reuse it if desired
np.random.seed(seed)   

rands = np.random.choice(np.arange(0,len(hc_ss[0,0])), 
                         size=nreals, replace=False, )
for rr, rand in enumerate(rands):
    cc = colors[rr]
    ax.plot(xx, bg[:,rand], alpha=0.5, zorder=6)
    ax.scatter(xx, hc_ss[:,rand,0], marker='o', color=cc, alpha=0.2,
               edgecolors='k', zorder=5)
    for ll in range(nloudest):
        ax.scatter(xx, hc_ss[:,rand,ll], marker='o', color=cc, alpha=0.2,
                   )
        
name = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/lunchtalks'
name += f'/hc_{seed}.png'
fig.savefig(name,
            transparent=True, bbox_inches='tight', dpi=100)

In [ ]:


fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN,
                     figsize=(6.5,4))
xx = data[10]['fobs_cents'] *YR
yy = xx**(-2/3)*1.1*10**-15

_, bg = holo.single_sources.resample_loudest(hc_ss, hc_bg, 1)
ax.plot(xx,yy, color='k', alpha=0.25, linestyle='dashed')

ax.plot(xx, np.median(bg, axis=-1), color='k')
for pp in [50,95]:
    lo, hi = np.percentile(bg, (50-pp/2, 50+pp/2), axis=-1)
    ax.fill_between(xx, lo, hi, color='k', alpha=0.2)


# ax.scatter(xx, np.max(hc_ss[:,0,:], axis=-1), color='k', alpha=0.2)

nreals = 5
nloudest = 3
cmap = plot.truncate_colormap('jet_r',0.1, 0.999)
colors = cmap(np.linspace(0,1,nreals))
colors = ['r',  'dodgerblue',
          'purple', 'g', 'orange']

seed = 7197
# seed = 89118
seed = 1811
# seed=None
if seed is None:
    seed = np.random.randint(99999)   # get a random number
    print(f"random seed: {seed}")                           # print it out so we can reuse it if desired
np.random.seed(seed)   

rands = np.random.choice(np.arange(0,len(hc_ss[0,0])), 
                         size=nreals, replace=False, )
for rr, rand in enumerate(rands):
    cc = colors[rr]
    ax.plot(xx, bg[:,rand], alpha=0.5, zorder=6)
    ax.scatter(xx, hc_ss[:,rand,0], marker='o', color=cc, alpha=0.2,
               edgecolors='k', zorder=5)
    for ll in range(nloudest):
        ax.scatter(xx, hc_ss[:,rand,ll], marker='o', color=cc, alpha=0.2,
                   )
        
name = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/lunchtalks'
name += f'/hc_{seed}.png'
fig.savefig(name,
            transparent=True, bbox_inches='tight', dpi=100)

In [ ]:
print(hc_ss[:,rands,:3].shape)

In [ ]:
filename = figloc+'/eli_strain.npz'

single_sources = np.swapaxes(hc_ss[:,rands,:3], 0,1)
print(single_sources.shape)
background = np.swapaxes(bg[:,rands], 0,1)
print(background.shape)
frequencies, _ = utils.pta_freqs()

info = ["get single sources as 'single_sources[realization, frequency, loudest]'",
        "get background as 'background[realization, frequency]'"]

np.savez(filename, 
         single_sources=single_sources, background=background, info=info,
         frequencies=frequencies)

# Varying h_c and Mass

In [ ]:
maps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10',
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner'
]

In [ ]:

for tt, target in enumerate(targets):
    # if tt != 3:
    #     continue
    colors=maps[tt](np.linspace(0,1,3))
    fontsize=16

    xlabel=plot.LABEL_GW_FREQUENCY_YR
    ylabels = ['$h_c$', '$M\ [\mathrm{M}_\odot]$']
    xx, yy_ss, yy_bg, parlabels = detstats.get_hcpar_arrays(
                target, nloudest=1, parvars=[0,2,4],
            )

    for ii in [1,0]:
        fig, ax = plot.figax(figsize=(4,2.5))
        if ii==0: ax.set_xlabel(xlabel, fontsize=fontsize)
        ax.set_ylabel(ylabels[ii], fontsize=fontsize)
        handles = []
        labels = []
        for vv in range(3):
            ax.plot(xx, np.median(yy_bg[vv,ii,:,:], axis=-1), color=greys[vv],
                        linestyle='-', alpha=0.85, zorder=2)
            pp=68
            med, *conf = np.percentile(yy_ss[vv,ii,:,:], (50, 50-pp/2, 50+pp/2), axis=-1)
            ax.plot(xx, med, color=colors[vv], linestyle='--', alpha=0.75, zorder=3)
            ax.fill_between(xx, *conf, color=colors[vv], alpha=0.2, zorder=1)
            handles.append(mpl.lines.Line2D([0], [0], color=colors[vv]))
            labels.append(parlabels[vv])
            
        
        fig.patch.set_facecolor('white')
        fig.patch.set_alpha(0.2)
        fig.savefig(figloc+f'/{target}_par_{ii}.png', transparent=False, bbox_inches='tight')

        han = [
            mpl.lines.Line2D([0], [0], linestyle='--', label='SS',
                            color=colors[1]),
            mpl.lines.Line2D([0], [0], linestyle='-', label='BG',
                            color=greys[1])
        ]
        ax.legend(handles=han, loc='lower left')
    fig,ax = plot.figax_single(height=2)
    ax.legend(handles=handles, labels=labels, ncols=3, title=plot.PARAM_KEYS[target],
            fancybox=True, framealpha=0.6, loc='center')
    fig.savefig(figloc+f'/{target}_par_leg.png', transparent=True, bbox_inches='tight')


# EVSS

In [ ]:
col0p0 = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]

col_gw = [
    '#595959', '#595959',
    '#595959', '#595959',
    '#595959', '#595959', 
]

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10',
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner'
]

In [ ]:
for tt, target in enumerate(targets):
    if True:
        fig,ax = plot.figax(xscale='linear', figsize=(5,3.5))
        ax.set_xlabel(plot.PARAM_KEYS[target], fontsize=18)
        ax.set_ylabel(plot.LABEL_EVSS, fontsize=18)

        xx, yy = detstats.get_ratio_arrays(target, bgl=1)
        yy *= 0.5

        for pp in [50, 95]:
            med, *conf = np.percentile(yy, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx, med, alpha=0.9, color=col0p0[tt])
            ax.fill_between(xx, *conf, color=col0p0[tt], alpha=0.25)


        fig.patch.set_facecolor('white')
        fig.patch.set_alpha(0.2)


        filename = figloc+f'/{target}_evss.png'
        fig.savefig(filename, transparent=False, facecolor=fig.get_facecolor(),
                    bbox_inches='tight')

## all EVSS

In [ ]:
INCLUDE_GW = True
BGL=1
ylabel = plot.LABEL_EVSS
targets = [
    'gsmf_phi0', 'mmb_mamp_log10', 'hard_time',
    'gsmf_mchar0_log10', 'mmb_scatter_dex', 'hard_gamma_inner']
col0p0 = [
    '#336948',  '#9e5c41',   '#2d839f', 
    '#336948',  '#9e5c41',   '#2d839f',
]

col_gw = [
    '#595959', '#595959', '#595959', 
    '#595959', '#595959', '#595959', 
]

fig, axs = plot.figax(nrows=2, ncols=3, sharey=True, sharex=False, xscale='linear',
                      figsize=(8.5,5))
fig.text( 0.05, 0.5, ylabel, va='center', ha='center', rotation='vertical',
         fontsize=18)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])

    xx, yy0p0, = detstats.get_ratio_arrays(targets[ii], bgl=BGL)
    yy0p0 *= 0.5

    col=col0p0[ii]
    for pp in [50, 95]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col0p0[ii])
        ax.fill_between(xx, *conf, color=col0p0[ii], alpha=0.25)

    if INCLUDE_GW:
        if ii%3 != 2:
            xx_gw, yy_gw = detstats.get_ratio_arrays(targets[ii],  gw_only=True, bgl=BGL)
            yy_gw *= 0.5
            print(f"{ii=}, {yy_gw.shape}")
        else:
            xx_gw = xx
            yy_gw = yy_gw[int(NVARS/2)]
            yy_gw = np.repeat(yy_gw, 21).reshape(NREALS*NSKIES, NVARS)
            yy_gw = np.swapaxes(yy_gw, 0, 1).reshape(NVARS, NREALS, NSKIES,)
            print(f"{ii=}, {yy_gw.shape}")

        for pp in [50]:
            med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx_gw, med, alpha=0.95, color=col_gw[ii], linestyle='-.')
            ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.3,)
        print(f"{ii=}, yy_gw.shape")

    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')       
plt.subplots_adjust(wspace=0.05, hspace=0.35)
fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.2)

savename = figloc+f'/evss_vs_var{NVARS}_wn_bgl{1}_p{NPSRS}.png'
fig.savefig(savename, dpi=100, bbox_inches='tight', facecolor=fig.get_facecolor())

In [ ]:
ylabel = plot.LABEL_EVSS
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
col0p0 = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]

col_gw = [
    '#595959', '#595959',
    '#595959', '#595959',
    '#595959', '#595959', 
]

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    xx, yy0p0, = detstats.get_ratio_arrays(targets[ii], bgl=BGL)
    yy0p0 *= 0.5

    col=col0p0[ii]
    for pp in [50, 95]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col0p0[ii])
        ax.fill_between(xx, *conf, color=col0p0[ii], alpha=0.25)

    if INCLUDE_GW:
        if ii<4:
            xx_gw, yy_gw = detstats.get_ratio_arrays(targets[ii],  gw_only=True, bgl=BGL)
            yy_gw *= 0.5
            print(f"{ii=}, {yy_gw.shape}")
        else:
            xx_gw = xx
            yy_gw = yy_gw[int(NVARS/2)]
            yy_gw = np.repeat(yy_gw, 21).reshape(NREALS*NSKIES, NVARS)
            yy_gw = np.swapaxes(yy_gw, 0, 1).reshape(NVARS, NREALS, NSKIES,)
            print(f"{ii=}, {yy_gw.shape}")

        for pp in [50]:
            med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx_gw, med, alpha=0.95, color=col_gw[ii], linestyle='-.')
            ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.3,)
        print(f"{ii=}, yy_gw.shape")

    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')       
plt.subplots_adjust(wspace=0.05, hspace=0.35)

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.2)

savename = figloc+f'/evss_single.png'
fig.savefig(savename, dpi=100, bbox_inches='tight', facecolor=fig.get_facecolor())


# DPSS

In [ ]:
ylabel = 'Detection Probability'

targets = [
    'gsmf_phi0', 'mmb_mamp_log10', 'hard_time',
    'gsmf_mchar0_log10', 'mmb_scatter_dex', 'hard_gamma_inner']

col_ss = [
    '#336948',  '#9e5c41',   '#2d839f', 
    '#336948',  '#9e5c41',   '#2d839f',
]

col_gw = [
    '#595959', '#595959', '#595959', 
    '#595959', '#595959', '#595959', 
]

col_bg = [
    '#193423','#5b3425', '#164150',
    '#193423', '#5b3425', '#164150',
]

col_gwss = [
    'grey', 'grey',
    'grey', 'grey',
    'grey', 'grey', 
]

col_gwbg = [
    'black', 'black',
    'black', 'black',
    'black', 'black', 
]

fig, axs = plot.figax(nrows=2, ncols=3, sharey=True, sharex=False, xscale='linear',
                             figsize=(8.5,5))
fig.text( 0.05, 0.5, ylabel, va='center', ha='center', rotation='vertical',
         fontsize=18)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])\

    # ax.yaxis.set_major_locator(mpl.ticker.LogLocator(numticks=999))
    # ax.yaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=999, subs=(.2, .4, .6, .8)))

    # get arrays
    xx, yy_ss, _, yy_bg = detstats.get_dp_arrays(targets[ii], red=False, cv=10, bgl=BGL)

    # plot single and background sources
    for pp in [50, 95]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy_ss, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col_ss[ii])
        ax.fill_between(xx, *conf, color=col_ss[ii], alpha=0.25)

        med, *conf = np.percentile(yy_bg, [50, 50-pp/2, 50+pp/2], axis=(1))
        ax.plot(xx, med, alpha=0.9, color=col_bg[ii], linestyle='--')
        ax.fill_between(xx, *conf, color=col_bg[ii], alpha=0.35)

    # add gw-only
    if ii%3 !=2:
        _, gw_ss, _, gw_bg = detstats.get_dp_arrays(targets[ii], red=False, gw_only=True, cv=10, bgl=BGL)       
        print(f"{gw_bg.shape=}")
        print(holo.utils.stats(np.std(gw_bg, axis=(1)))) 
        # print(f"{ii=}, {yy_gw.shape}")
    else:
        gw_ss = gw_ss[int(NVARS/2)]
        gw_ss = np.repeat(gw_ss, 21).reshape(NREALS*NSKIES, NVARS)
        gw_ss = np.swapaxes(gw_ss, 0, 1).reshape(NVARS, NREALS, NSKIES,)

        gw_bg = gw_bg[int(NVARS/2)]
        gw_bg = np.repeat(gw_bg, 21).reshape(NREALS, NVARS)
        gw_bg = np.swapaxes(gw_bg, 0, 1).reshape(NVARS, NREALS)
        print(f"{ii=}, {gw_ss.shape=}")
        print(f"{gw_bg.shape=}")

    for pp in [50]:
        med, *conf = np.percentile(gw_ss, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.95, color=col_gwss[ii], linestyle='-.')
        ax.fill_between(xx, *conf, color=col_gwss[ii], alpha=0.3)

        med, *conf = np.percentile(gw_bg, [50, 50-pp/2, 50+pp/2], axis=(1))
        ax.plot(xx, med, alpha=0.6, color=col_gwbg[ii], linestyle=':', lw=2)
        ax.fill_between(xx, *conf, color=col_gwbg[ii], alpha=0.25)

    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')   
    
plt.subplots_adjust(wspace=0.05, hspace=0.35)
# fig.tight_layout()

savename = figloc+f'/dpboth_double.png'
fig.savefig(savename, dpi=100, bbox_inches='tight')

In [ ]:
ylabel = 'Detection Probability'
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
col_ss = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]

col_bg = [
    '#193423', '#193423', 
    '#5b3425', '#5b3425', 
    '#164150', '#164150',
]

col_gwss = [
    'grey', 'grey',
    'grey', 'grey',
    'grey', 'grey', 
]

col_gwbg = [
    'black', 'black',
    'black', 'black',
    'black', 'black', 
]

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    # ax.yaxis.set_major_locator(mpl.ticker.LogLocator(numticks=999))
    # ax.yaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=999, subs=(.2, .4, .6, .8)))

    # get arrays
    xx, yy_ss, _, yy_bg = detstats.get_dp_arrays(targets[ii], red=False, cv=10, bgl=BGL)

    # plot single and background sources
    for pp in [50, 95]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy_ss, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col_ss[ii])
        ax.fill_between(xx, *conf, color=col_ss[ii], alpha=0.25)

        med, *conf = np.percentile(yy_bg, [50, 50-pp/2, 50+pp/2], axis=(1))
        ax.plot(xx, med, alpha=0.9, color=col_bg[ii], linestyle='--')
        ax.fill_between(xx, *conf, color=col_bg[ii], alpha=0.35)

    # add gw-only
    if ii<4:
        _, gw_ss, _, gw_bg = detstats.get_dp_arrays(targets[ii], red=False, gw_only=True, cv=10, bgl=BGL)       
        print(f"{gw_bg.shape=}")
        print(holo.utils.stats(np.std(gw_bg, axis=(1)))) 
        # print(f"{ii=}, {yy_gw.shape}")
    else:
        gw_ss = gw_ss[int(NVARS/2)]
        gw_ss = np.repeat(gw_ss, 21).reshape(NREALS*NSKIES, NVARS)
        gw_ss = np.swapaxes(gw_ss, 0, 1).reshape(NVARS, NREALS, NSKIES,)

        gw_bg = gw_bg[int(NVARS/2)]
        gw_bg = np.repeat(gw_bg, 21).reshape(NREALS, NVARS)
        gw_bg = np.swapaxes(gw_bg, 0, 1).reshape(NVARS, NREALS)
        print(f"{ii=}, {gw_ss.shape=}")
        print(f"{gw_bg.shape=}")

    for pp in [50]:
        med, *conf = np.percentile(gw_ss, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.95, color=col_gwss[ii], linestyle='-.')
        ax.fill_between(xx, *conf, color=col_gwss[ii], alpha=0.3)

        med, *conf = np.percentile(gw_bg, [50, 50-pp/2, 50+pp/2], axis=(1))
        ax.plot(xx, med, alpha=0.6, color=col_gwbg[ii], linestyle=':', lw=2)
        ax.fill_between(xx, *conf, color=col_gwbg[ii], alpha=0.25)

    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')   
    
plt.subplots_adjust(wspace=0.05, hspace=0.35)
# fig.tight_layout()


fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.2)

savename = figloc+f'/dpboth_single.png'
fig.savefig(savename, dpi=100, bbox_inches='tight', facecolor=fig.get_facecolor())

In [ ]:
fig, ax = plot.figax(figsize=(2,1))
handles = [
    mpl.lines.Line2D([0], [0], linestyle='-', label='SS', color='k'),
    mpl.lines.Line2D([0], [0], linestyle='--', label='BG', color='k')
]
ax.legend(handles=handles, 
            fancybox=True, framealpha=0.6, loc='center' )
name = figloc+'/dplegend.png'

fig.savefig(name, transparent=True)

# Schechter Function

In [ ]:

mass = np.logspace(6, 13, 91)*MSOL
colors= cmap_Greens(np.linspace(0,1,3))


fig,ax = plot.figax(figsize=(4,3))
ax.set_xlabel('$M\ [\mathrm{M}_\odot]$', fontsize=16)
ax.set_ylabel('$\Psi \equiv \\frac{d \eta_*}{d\log_{10} m_*}$', fontsize=20)

xx = mass/MSOL

handles = []
labels = []
for ii, mchar0_log10 in enumerate([10.5, 11.5, 12.5]):
    gsmf = holo.sams.comps.GSMF_Schechter(phi0=-2.5, mchar0_log10=mchar0_log10)
    phi = gsmf(mass, redz=1)
    hh, = ax.plot(xx, phi, color=colors[ii], alpha=0.9)
    handles.append(hh)

    ax.axvline(10**mchar0_log10, linestyle='--', color=colors[ii], alpha=0.5)
    labels.append(f'{mchar0_log10}')

ax.set_ylim(10**-8, 10**0)
ax.legend(handles=handles, labels=labels, title=plot.PARAM_KEYS['gsmf_mchar0_log10'],
          loc='lower left', ncols=1, fancybox=True, framealpha=0.2,)
fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.0)

filename = figloc+'/gsmf_func_mchar0.png'
fig.savefig(filename, transparent=False, bbox_inches='tight')

In [ ]:

mass = np.logspace(6, 13, 91)*MSOL
colors= cmap_Greens(np.linspace(0,1,3))


fig,ax = plot.figax(figsize=(4,3))
ax.set_xlabel('$M\ [\mathrm{M}_\odot]$', fontsize=16)
ax.set_ylabel('$\Psi \equiv \\frac{d \eta_*}{d\log_{10} m_*}$', fontsize=20)

xx = mass/MSOL

handles = []
labels = []
for ii, phi0 in enumerate([-3.5, -2.5, -1.5]):
    gsmf = holo.sams.comps.GSMF_Schechter(phi0=phi0, mchar0_log10=11.5)
    phi = gsmf(mass, redz=1)
    hh, = ax.plot(xx, phi, color=colors[ii], alpha=0.9)
    handles.append(hh)

    ax.axhline(10**phi0, linestyle='--', color=colors[ii], alpha=0.5)
    labels.append(f'{phi0}')

ax.set_ylim(10**-8, 10**0)
ax.legend(handles=handles, labels=labels, title=plot.PARAM_KEYS['gsmf_phi0'],
          loc='lower left', ncols=3, fancybox=True, framealpha=0.2,)

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.2)

filename = figloc+'/gsmf_func_phi0.png'
fig.savefig(filename, transparent=False, bbox_inches='tight')

# Separation Evolution

In [ ]:
fileloc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09B/'
filename = fileloc+f'evol_{target}_rv.npz'
file = np.load(filename, allow_pickle=True)
rv_hard_gamma_inner = file['rv']
file.close()

In [ ]:
NSTEPS = 20

def construct_evolution(target_param, params, nsteps, 
    pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=None, seed=None),
):
    """ Construct evolution data.

    saves
    -----
    taus : [nparams][nmasses] array of [nsteps] 1Darrays
        hardening timescales
    target_param_list : [nparams] 
        each of the target's values


    """
    m1_range = [10**7.5*MSOL, 10**8.5*MSOL] #1e8
    m2_range = [10**9.5*MSOL, 10**10.5*MSOL] #1e10
    mtot_ranges = [m1_range, m2_range]
    mrat_range = [0.2, 1.0]
    redz_range = [0, np.inf]

    # Whatever param we're varying
    target_param_list =  []
    # range of binary separations to plot
    sepa = np.logspace(-3, 3, nsteps)[::-1] * PC

    taus1 = [] # shape [nvars]
    taus2 = [] # shape [nvars]
    
    freq1 = []
    freq2 = []

    # Iterate over target lifetimes
    for tt in tqdm(range(len(params))):

        # using my parameters from above
        _params = params[tt] # midpoints
        target_param_list.append(_params[target_param])

        sam, hard = pspace.model_for_params(_params)

        # call static_binary_density property to setup variables
        _ = sam.static_binary_density 

        # calculate binary properties at target separations
        _edges, _dnum, _redz_final, _details = sam._dynamic_binary_number_at_sepa_consistent(
            hard, sepa, details=True) # it would be better if I saved these details when I first calculated them!
        
        # select the bins with target binary parameters
        _taus = [] # Shape [nmassses,]
        _freq = []
        for mtot_range in mtot_ranges:
            sel_mtot  = (mtot_range[0] < sam.mtot) & (sam.mtot <= mtot_range[1])
            sel_mrat = (mrat_range[0] < sam.mrat) & (sam.mrat <= mrat_range[1])
            sel_redz = (redz_range[0] < sam.redz) & (sam.redz <= redz_range[1])
            sel = (
                sel_mtot[:, np.newaxis, np.newaxis] *
                sel_mrat[np.newaxis, :, np.newaxis] * 
                sel_redz[np.newaxis, np.newaxis, :]
            )
            _taus.append(_details['tau'][sel].T)
            masses = sam.mtot[sel_mtot]
            print(f"{masses.shape=}, {sepa.shape=}")
            _freq.append(utils.kepler_freq_from_sepa(sam.mtot[sel_mtot,np.newaxis],  
                                                    sepa[np.newaxis,:]))
        print(f"{_taus[0].shape=}")   
            
        taus1.append(_taus[0])
        taus2.append(_taus[1])
        freq1.append(_freq[0])
        freq2.append(_freq[1])

    # save results
    fileloc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09B/'
    filename = fileloc+f'evol_lunchtalk.npz' 

    np.savez(filename, target_param_list=target_param_list,
        nsteps=nsteps, sepa=sepa,
        mtot_ranges = mtot_ranges,
        taus1=taus1, 
        taus2=taus2, 
        freq1=freq1, # M, nsteps
        freq2=freq2,)
    
    rv = dict(
        taus1=taus1, taus2=taus2, 
        freq1=freq1, freq2=freq2, 
        target_list=target_param_list, nsteps=nsteps, sepa=sepa,
        mtot_ranges=mtot_ranges)
    print(rv)
    return rv

In [ ]:
target='hard_gamma_inner'
 
_, params = detstats.get_data(target, dets=False)
params = [params[0], params[10], params[20]]
fobs_gw_cents = data[0]['fobs_cents']
_, fobs_gw_edges = holo.utils.pta_freqs()
rv_hard_gamma_inner = construct_evolution(target_param=target, params=params, nsteps=NSTEPS)



In [ ]:

m1_range = [10**7.5*MSOL, 10**8.5*MSOL] #1e8
m2_range = [10**9.5*MSOL, 10**10.5*MSOL] #1e10
mtot_ranges = [m1_range, m2_range,]

In [ ]:
rv = rv_hard_gamma_inner
fontsize=14

# Make figure
fig, ax = plot.figax_single(figsize=(5,3.2))
xlabel2 = "Frequency ($\mathrm{yr}^{-1}$)"
ax.set_xlabel(xlabel2, fontsize=fontsize)
ax.set_ylabel(plot.LABEL_HARDENING_TIME, fontsize=fontsize)

x1 = rv['sepa'] # S
yys = [rv['taus1'], rv['taus2'],] # [2masses, 3vars, M, Q, Z]

mtots = 10**np.mean(np.log10(mtot_ranges), axis=-1)

mass_handles = []
mass_labels = []

target_handles = []
target_labels = []

linestyles=[':', '-']
colors = cmap_Blues(np.linspace(0,1,3))

for vv, targ in enumerate(rv['target_list']):
    for mm, mtot in enumerate(mtots):
        x2 = utils.kepler_freq_from_sepa(np.ones(len(x1))*mtot, x1) # frequencies
        # hh, gg = plot.draw_med_conf_color(axs[0], x1/PC, yys[mm][vv]/GYR, 
        #                             color=colors[mm][vv], linestyle=linestyles[vv], fracs=[0.5,])
        hh, gg = plot.draw_med_conf_color(ax, x2*YR, yys[mm][vv]/GYR, 
                                    color=colors[vv], linestyle=linestyles[mm], fracs=[0.5,])
        if vv==1: 
            mass_handles.append(
                mpl.lines.Line2D([0], [0], color='k', linestyle=linestyles[mm])
            )
            mass_labels.append(f"~{mtot/MSOL:.1e} $M_\odot$")
        if mm==1:
            target_handles.append(hh)
            target_labels.append(targ)
ax.axvline(data[0]['fobs_cents'][0]*YR, linestyle='-', color='k', alpha=0.5)
ax.axvline(data[0]['fobs_cents'][-1]*YR, linestyle='-', color='k', alpha=0.5)

mass_labels = ['~$10^8 \mathrm{M}_\odot$', '~$10^{10} \mathrm{M}_\odot$']
leg1 = ax.legend(handles=mass_handles, labels=mass_labels, fontsize=12, title='Mass',
                 bbox_to_anchor=(0., 0.27), loc='lower left', ncols=2,
                 columnspacing=1)
leg2 = ax.legend(handles=target_handles, labels=target_labels, fontsize=12, title=plot.PARAM_KEYS[target], 
                 loc='lower left', ncols=3, columnspacing=1)

ax.add_artist(leg1)

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.2)

filename = figloc+f'/freq_evolution.png'
fig.savefig(filename, transparent=False, facecolor=fig.get_facecolor(),
            bbox_inches='tight')

# Healmaps

In [ ]:
target = 'hard_time'
data, params = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=1000, nexcl=0, dets=False, nvars=3
        )
hc_ss = data[1]['hc_ss']
hc_bg = data[1]['hc_bg']
moll, Cl = holo.anisotropy.sph_harm_from_hc(hc_ss[0:1,0:50,:], hc_bg[0:1,0:50], 
                                            nside=32, lmax=8)

In [ ]:
ff=0
rr=7
fig,ax = plot.figax(figsize=(7,5))
hp.mollview(moll[ff,rr],  hold=True, title=None)
fig.text(0.5,0.12, '$h_c^2/d\Omega$', va='center', ha='center')

# fig, ax = plt.gcf(), plt.gca()
fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.00)

savename = figloc+f'/hp_r{rr}.png'
fig.savefig(savename, dpi=100, bbox_inches='tight', transparent=False, facecolor=fig.get_facecolor())


In [ ]:
lmax=8

seed=91829
if seed is None:
    seed = np.random.randint(99999)   # get a random number
    print(f"random seed: {seed}")                           # print it out so we can reuse it if desired
np.random.seed(seed)   

moll_recon = hp.synfast(Cl[ff,rr], nside=32, lmax=lmax)

fig,ax = plot.figax(figsize=(5,3))
hp.mollview(moll_recon,  hold=True, cbar=False, 
            title='$\ell_\mathrm{max}=%d$'%lmax)
# fig.text(0.5,0.12, '$h_c^2/d\Omega$', va='center', ha='center')

# fig, ax = plt.gcf(), plt.gca()
fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.00)

savename = figloc+f'/hp_r{rr}_l{lmax}.png'
fig.savefig(savename, dpi=100, bbox_inches='tight', 
            transparent=False, facecolor=fig.get_facecolor(),)

fig


In [ ]:
print(moll.shape, Cl.shape)

blank map

In [ ]:
ff=0
rr=100
new_Cl = np.zeros_like(Cl[ff,rr])
new_Cl[0] = Cl[ff,rr,0]
Cl_map = hp.synfast(new_Cl, nside=8, lmax=8)
hp.mollview(Cl_map, title=None, cbar=False, hold=True)
fig, ax = plt.gcf(), plt.gca()
fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.00)

savename = figloc+f'/temp.png'
fig.savefig(savename, dpi=100, bbox_inches='tight', transparent=False, facecolor=fig.get_facecolor())

ClC0

In [ ]:
print(Cl.shape)

In [ ]:
xx = np.arange(0,len(Cl[ff,rr]))
yy = Cl[ff,rr,1:]/Cl[ff,rr,0]

fig, ax = plot.figax(figsize=(5,3.2), xlabel='$\ell$', ylabel='$C_\ell$',
                     xscale='linear')
ax.plot(xx, Cl[ff,rr])

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.02)

savename = figloc+f'/Clvsl.png'
fig.savefig(savename, dpi=100, bbox_inches='tight', transparent=False, facecolor=fig.get_facecolor())
fig       

# Anisotropy Plots

In [ ]:
fobs_cents, _ = utils.pta_freqs()

In [ ]:
ylabel = "$C_{\ell>0}/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
col0p0 = [
    '#5bb96a', '#5bb96a', 
    '#b98046', '#b98046', 
    '#4680b9', '#4680b9',
]
freq_idx = [0,1,2,3,4]

gnuplot_map = plot.truncate_colormap('gnuplot_r', 0.05, 0.9)
colors = gnuplot_map(np.linspace(0,1,5))
print(colors)
col3 = mpl.colors.to_rgba('#9f1d93')
print(col3)
colors[3] = col3
# colors = plasma_map(np.linspace(0,1,5))

colors = np.flip(colors, axis=0)

labels_fobs=[]
handles=[]
fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    xx, yy, _ = detstats.get_anis_var_arrays(targets[ii])
    for ff, fidx in enumerate(freq_idx):
        if ii==0: labels_fobs.append(f"{fobs_cents[fidx]*YR:.2f}")

        for pp in [68,]:
        # for pp in [50]: 
            med, *conf = np.percentile(yy[:,fidx,:], [50, 50-pp/2, 50+pp/2], axis=-1)
            # if ff%2 == 0: 
            ax.fill_between(xx, *conf, color=colors[ff], alpha=0.25, zorder=6-ii)
        hh, = ax.plot(xx, med, alpha=0.9, color=colors[ff], zorder=12-ii)
        handles.append(hh)
    
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')    
plt.subplots_adjust(wspace=0.05, hspace=0.35)

fig.legend(handles=handles, labels=labels_fobs, bbox_to_anchor = (0.54, -0.02), loc='lower center',
           ncols = len(freq_idx), title=plot.LABEL_GW_FREQUENCY_YR, title_fontsize=12, 
           columnspacing=1.25)

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.2)

savename = figloc+f'/anis_var_single.png'
fig.savefig(savename, dpi=100, bbox_inches='tight', facecolor=fig.get_facecolor())

In [ ]:
Cl_nihan = np.array([
    [0.20216773, 0.14690035, 0.09676646, 0.07453352, 0.05500382, 0.03177427],
    [0.21201336, 0.14884939, 0.10545698, 0.07734305, 0.05257189, 0.03090662],
    [0.20840993, 0.14836757, 0.09854803, 0.07205384, 0.05409881, 0.03305785],
    [0.19788951, 0.15765126, 0.09615489, 0.07475364, 0.0527356 , 0.03113331],
    [0.20182648, 0.14745265, 0.09681202, 0.0746824 , 0.05503161, 0.0317012 ]])
print(Cl_nihan.shape)

freq_bins_Hz = np.array([2.0, 4.0, 5.9, 7.9, 9.9]) *10**-9 # Hz

xx_nihan = freq_bins_Hz * YR # Hz
color_nihan = '#845b98'

In [ ]:
BUILD_ARRAYS=False
INCLUDE_LIMS = True
cmap_Purples = plot.truncate_colormap('Purples', 0.4, 1.0)

xlabel = plot.LABEL_GW_FREQUENCY_YR
xlabel2 = plot.LABEL_GW_FREQUENCY_NHZ
ylabel = r"$C_{\ell>0}/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
colors_purple = cmap_Purples(np.linspace(0,1,6))
parvars = [0,10,20]

fig, axs = plot.figax_single(ncols=2, nrows=3, height=7,
                             sharey=True, sharex=True)
fig.text(0.55, 0.08, xlabel, va='center', ha='center')
fig.text(0.55, 1.02, xlabel2, va='center', ha='center')
fig.text(0.05, 0.55, ylabel, va='center', ha='center', rotation='vertical')

for ii, ax in enumerate(axs.flatten()):
    target = targets[ii]
    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = detstats.build_anis_freq_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST,
        nvars=NVARS, nreals=NREALS)
    else:
        yy_cl, xx_fobs, params_cl = detstats.get_anis_freq_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST)

    xx = xx_fobs*YR
    xx = xx_fobs*YR

    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

    # add nloudest=10, for comparison
    yy_cl, _, params_cl = detstats.get_anis_freq_arrays(targets[ii], parvars=[0,10,20], nloudest=10)
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv, linestyle=':')

    if INCLUDE_LIMS:
        for ll, ell in enumerate(np.arange(1,7)):
            ax.plot(xx_nihan, Cl_nihan[:,ll], marker='o', linestyle='--',
                    color=colors_purple[ll], markersize=4, zorder=3.5)

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    # ax.tick_params(axis='x', which='both', top=True, direction='in')       
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')

for ii, ax in enumerate(axs.flatten()[:2]):
    tw = ax.twiny()
    tw.grid(False)
    xlim = np.array(ax.get_xlim()) / YR * 1e9
    tw.set(xlim=xlim, xscale=ax.get_xscale())

    
fig.subplots_adjust(wspace=0, hspace=0)


fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.2)

savename = figloc+f'/anis_freq_single.png'
fig.savefig(savename, dpi=100, bbox_inches='tight', facecolor=fig.get_facecolor())
fig

In [ ]:
BUILD_ARRAYS=False
INCLUDE_LIMS = True
PARVARS=[0,1,2]
cmap_Purples = plot.truncate_colormap('Purples', 0.4, 1.0)
fontsize=14

xlabel = plot.LABEL_GW_FREQUENCY_YR
xlabel2 = plot.LABEL_GW_FREQUENCY_NHZ
ylabel = r"$C_{\ell>0}/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
colors_purple = cmap_Purples(np.linspace(0,1,6))
parvars = [0,10,20]


fig, axs = plot.figax(figsize=(8,3.5), ncols=2)
fig.text(0.5, -0.02, xlabel, fontsize=fontsize, ha='center', va='center')
for ii, target in enumerate(targets):
    if ii ==0:
        ax=axs[0]
        ax.set_ylabel(ylabel, fontsize=fontsize)
    elif ii==4:
        ax=axs[1]
        ax.set_yticklabels([])
        ax.set_ylim(axs[0].get_ylim())
    else:
        continue


    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = detstats.build_anis_freq_arrays(targets[ii], parvars=PARVARS, nloudest=1000,
        nvars=NVARS, nreals=NREALS)
    else:
        yy_cl, xx_fobs, params_cl = detstats.get_anis_freq_arrays(targets[ii], parvars=PARVARS, nloudest=1000)

    xx = xx_fobs*YR
    xx = xx_fobs*YR

    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

    # add nloudest=10, for comparison
    yy_cl, _, params_cl = detstats.get_anis_freq_arrays(targets[ii], parvars=[0,10,20], nloudest=10)
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv, linestyle=':')

    if INCLUDE_LIMS:
        for ll, ell in enumerate(np.arange(1,7)):
            ax.plot(xx_nihan, Cl_nihan[:,ll], marker='o', linestyle='--',
                    color=colors_purple[ll], markersize=4, zorder=3.5)

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    # ax.tick_params(axis='x', which='both', top=True, direction='in')       
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')

fig.subplots_adjust(wspace=0)
fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.2)

savename = figloc+f'/phi0_and_thard_anis_freq.png'
fig.savefig(savename, dpi=100, bbox_inches='tight', facecolor=fig.get_facecolor())
fig


In [ ]:
BUILD_ARRAYS=False
INCLUDE_LIMS = True
PARVARS=[0,1,2]
cmap_Purples = plot.truncate_colormap('Purples', 0.4, 1.0)

xlabel = plot.LABEL_GW_FREQUENCY_YR
xlabel2 = plot.LABEL_GW_FREQUENCY_NHZ
ylabel = r"$C_{\ell>0}/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
colors_purple = cmap_Purples(np.linspace(0,1,6))
parvars = [0,10,20]


for ii, target in enumerate(targets):
    if ii != 0 and ii!=4:
        continue
    fig, ax = plot.figax(figsize=(4,3))
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = detstats.build_anis_freq_arrays(targets[ii], parvars=PARVARS, nloudest=1000,
        nvars=NVARS, nreals=NREALS)
    else:
        yy_cl, xx_fobs, params_cl = detstats.get_anis_freq_arrays(targets[ii], parvars=PARVARS, nloudest=1000)

    xx = xx_fobs*YR
    xx = xx_fobs*YR

    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

    # add nloudest=10, for comparison
    yy_cl, _, params_cl = detstats.get_anis_freq_arrays(targets[ii], parvars=[0,10,20], nloudest=10)
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv, linestyle=':')

    if INCLUDE_LIMS:
        ll=0
        ax.plot(xx_nihan, Cl_nihan[:,ll], marker='o', linestyle='--',
                    color=colors_purple[3], markersize=4, zorder=3.5)

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    # ax.tick_params(axis='x', which='both', top=True, direction='in')       
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')

    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(0.2)

    savename = figloc+f'/{target}_anis_freq.png'
    fig.savefig(savename, dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())
    fig


In [ ]:
ylabel = "$C_{1}/C_0$"
targets = [
    'gsmf_phi0', 
      'mmb_mamp_log10', 
      'hard_time', 
    'gsmf_mchar0_log10',  
      'mmb_scatter_dex',
        'hard_gamma_inner']
col0p0 = [
    '#5bb96a',
    '#b98046',
    '#b98046', 
      '#5bb96a',  
    '#4680b9', '#4680b9',
]
freq_idx = [0,1,2,3,4]
# colors = jet_map(np.linspace(0,1,len(freq_idx)))
gnuplot_map = plot.truncate_colormap('gnuplot_r', 0.05, 0.9)
fobs_cents, _ = utils.pta_freqs()

colors = ['#ec0d47', 
          '#ec6a3a', 
          '#e0aa4c', 
          '#a7d46b', 
          '#17a572']
colors = ['#a02c5d', '#ec6a3a', '#e0aa4c', '#17a572', '#032c7a']
colors = ['#ef5465', '#fecd54', '#7da855', '#4fc1e8', '#ac92eb' ]
colors = gnuplot_map(np.linspace(0,1,5))
print(colors)
col3 = mpl.colors.to_rgba('#9f1d93')
print(col3)
colors[3] = col3
# colors = plasma_map(np.linspace(0,1,5))

colors = np.flip(colors, axis=0)

labels_fobs=[]
handles=[]
fig, axs = plot.figax(nrows=2, ncols=3, sharey=True, sharex=False, xscale='linear',
                             figsize=(8,5))
fig.text(0.07, 0.5, ylabel, va='center', ha='center', rotation='vertical')
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    xx, yy, _ = detstats.get_anis_var_arrays(targets[ii])
    for ff, fidx in enumerate(freq_idx):
        if ii==0: labels_fobs.append(f"{fobs_cents[fidx]*YR:.2f}")

        for pp in [68,]:
        # for pp in [50]: 
            med, *conf = np.percentile(yy[:,fidx,:], [50, 50-pp/2, 50+pp/2], axis=-1)
            # if ff%2 == 0: 
            ax.fill_between(xx, *conf, color=colors[ff], alpha=0.25, zorder=6-ii)
        hh, = ax.plot(xx, med, alpha=0.9, color=colors[ff], zorder=12-ii)
        handles.append(hh)

        ax.axhline(Cl_nihan[ff,0], linestyle='--', color='tab:blue')
    
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')    
plt.subplots_adjust(wspace=0.05, hspace=0.35)

leg = fig.legend(handles=handles, labels=labels_fobs, bbox_to_anchor = (0.91, 0.5), loc='center left',
           ncols = 1, title='$f\ (\mathrm{yr}^{-1})$ ', title_fontsize=12, 
           columnspacing=1.25,)

# handles = [
#     mpl.lines.Line2D([0], [0], linestyle='--', color='tan:blue', label='Bayesian limit')
# ]


fig.subplots_adjust(wspace=0)
fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.2)

save_name = figloc+'/anis_v21_first5bins_lims_sidelegend.png'
fig.savefig(save_name, dpi=200, bbox_inches='tight', facecolor=fig.get_facecolor())